In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
import os,glob

In [4]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 8.2MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3


In [5]:
!pip install tensorflow


     |████████████████████████████████| 165.1 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 8.3 MB/s eta 0:00:01    |█████████████████████████▎      | 8.4 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 12.2 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 42 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 3.4 MB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 127 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 5.5

In [15]:
!pip install Keras

In [144]:
import tensorflow as tf

from tensorflow import keras 
from tensorflow.keras import layers
from keras.models import model_from_yaml
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array

# 1. Data Processing: 

In [145]:
#   a) Use the "ImageDataGenerator()" class from keras.processing.image to build out an instance called "train_datagen" with the following parameters: 

train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)

In [146]:
# b) Then build your training set by using the method ".flow_from_directory()"

training_set = train_datagen.flow_from_directory(
    directory = '/Users/mohammedhussain/Desktop/UCHICAGO assigments/Machine learning/assignment 7 nov 20/dataset_train',
    target_size = (64,64),
    batch_size = 32,
    class_mode = "categorical",
    seed = 66)

Found 88 images belonging to 4 classes.


So train_datagen.flow_from_directory has found around 88 images from 4 different classes or categories

In [147]:
# getting the count of all images from each category
training_set_files = glob.glob("dataset_train/*")
count_images = {}


In [148]:
# Iterating over each image in the trainning image set in ech category
for i in training_set_files:
    images_by_category = len(os.listdir(i+'/'))
    count_images[i] = images_by_category
count_images

{'dataset_train/category 2': 22,
 'dataset_train/category 4': 22,
 'dataset_train/category 3': 22,
 'dataset_train/category 1': 22}

There are 4 different categories or classes and each category/class , there are 22 images

In [149]:
for i in training_set_files:
    train_images = list(os.listdir(i+'/'))
train_images

['1031.png',
 '1025.png',
 '1024.png',
 '1030.png',
 '1032.png',
 '1033.png',
 '1023.png',
 '1020.png',
 '1021.png',
 '1052.png',
 '1051.png',
 '1050.png',
 '1040.png',
 '1041.png',
 '1043.png',
 '1042.png',
 '1010.png',
 '1011.png',
 '1013.png',
 '1012.png',
 '1000.png',
 '1014.png']

In [150]:
# lets take a random sample image from the above list
sample_image = image.load_img('dataset_train/category 1/1013.png')
x_image = img_to_array(sample_image)
x_image.shape

(800, 800, 3)

The shape of each training image observation is (800,800,3). This is a raw image size , not the input image size to the CNN classifer

In [151]:
# shape of the image after processing
training_set.target_size

(64, 64)

In [ ]:
If we look at the actual shapes of the image in the training set , we come across very strange images (chinese symbols,
 skulls, helmet  cross symbols, star wars helmet etc)                                                                                                     )

                                            

(64,64,3) will be our input to the CNN image classifer

# 2. Initial Classifier Build: 

In [152]:
classifier = Sequential()
classifier.add(Conv2D(filters = 32,kernel_size =(3,3), input_shape = (64,64,3),activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Conv2D(filters = 64,kernel_size =(3,3),activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Flatten()) #     converting the feature maps to 1D feature vectors
classifier.add(Dense(units = 128,activation = "relu"))
classifier.add(Dense(units = 4,activation = "softmax"))
classifier.compile(optimizer = 'adam',loss ='categorical_crossentropy',metrics = ['accuracy'])
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dense_5 (Dense)              (None, 4)                

# 3. Model Runs: 

In [153]:
classifier.fit(training_set,steps_per_epoch =3,epochs = 3)

Epoch 1/3
3/3 [==============================] - 1s 177ms/step - loss: 1.7567 - accuracy: 0.2727
Epoch 2/3
3/3 [==============================] - 1s 185ms/step - loss: 1.2310 - accuracy: 0.5795
Epoch 3/3
3/3 [==============================] - 0s 155ms/step - loss: 0.7861 - accuracy: 0.7955


In [154]:
# Saving Model and its weights to a file
my_model = classifier.to_yaml()
with open("My_Model.yaml","w") as file:
    file.write(my_model)
classifier.save('My_Model.h5')
print("Saved model")

Saved model


In [155]:
# Predicting on the test data images using the model we already built

# returns a compiled model
# identical to the previous one
model = load_model('My_Model.h5')
print("Loaded model from disk")

Loaded model from disk


In [156]:
# test data path
img_dir = "dataset_test/"

In [157]:
# iterate over each test image
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

In [158]:
# print the files in the dataset_test folder 
for f in files:
    print(f)

dataset_test/C033.png
dataset_test/1022.png
dataset_test/4011.png
dataset_test/1053.png
dataset_test/6051.png
dataset_test/4053.png
dataset_test/C014.png
dataset_test/6023.png


In [160]:
# make a prediction and add to results 
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results


[array([3]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([2]),
 array([1]),
 array([1])]

In [161]:
 # d) Determine accuracy.

# check category labels in training_set

training_set.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

In [162]:
 # First we need to understand the sequence of how the images were read in the test dataset directory
for f1 in files :
    print(f1)

dataset_test/C033.png
dataset_test/1022.png
dataset_test/4011.png
dataset_test/1053.png
dataset_test/6051.png
dataset_test/4053.png
dataset_test/C014.png
dataset_test/6023.png


In [163]:
# Manual labelling
test_labels= [3, 0, 2, 0, 1, 2, 3, 1]

In [164]:
# Calculating accuracy by comparing the predicted values to the actual values for the test set and calculate accuracy score
accuracy = metrics.accuracy_score(test_labels,results)
accuracy

0.75

e) Run this process for the following combinations:

* (steps_per_epoch: 1, epochs: 1)
* (steps_per_epoch: 1, epochs: 2)
* (steps_per_epoch: 1, epochs: 3)
* (steps_per_epoch: 2, epochs: 4)
* (steps_per_epoch: 2, epochs: 5)
* (steps_per_epoch: 2, epochs: 6)
* (steps_per_epoch: 3, epochs: 7)
* (steps_per_epoch: 3, epochs: 8)
* (steps_per_epoch: 5, epochs: 9)
* (steps_per_epoch: 5, epochs: 10)

In [165]:
parameter_grid = [
 {"model_name": "model_1", "steps_per_epoch": 1, "epochs": 1},
 {"model_name": "model_2", "steps_per_epoch": 1, "epochs": 2},
 {"model_name": "model_3", "steps_per_epoch": 1, "epochs": 3},
 {"model_name": "model_4", "steps_per_epoch": 2, "epochs": 4},
 {"model_name": "model_5", "steps_per_epoch": 2, "epochs": 5},
 {"model_name": "model_6", "steps_per_epoch": 2, "epochs": 6},
 {"model_name": "model_7", "steps_per_epoch": 3, "epochs": 7},
 {"model_name": "model_8", "steps_per_epoch": 3, "epochs": 8},
 {"model_name": "model_9", "steps_per_epoch": 5, "epochs": 9},
 {"model_name": "model_10", "steps_per_epoch": 5,"epochs": 10}
]

In [166]:
# Creating the image classifier model to fit and save the weights to the file/disk
def model_fit_save(model_params):
    print(f"Fitting the model: {model_params}\n\n")
    classifier_fit = classifier.fit(train_datagen.flow_from_directory(
    directory = '/Users/mohammedhussain/Desktop/UCHICAGO assigments/Machine learning/assignment 7 nov 20/dataset_train',
    target_size = (64,64),
    batch_size = 32,
    class_mode = "categorical"), steps_per_epoch=model_params["steps_per_epoch"], epochs=model_params["epochs"])
    # Saving the model and its weights as a file to the disk
    classifier_yaml = classifier.to_yaml()
    classifier_name = model_params["model_name"]
    with open(f"{classifier_name}.yaml", "w") as f:
        f.write(classifier_yaml)
        classifier.save(f"{classifier_name}.h5")
        print("Saving model to disk as a file\n")

In [167]:
# Creating  function to predict on the test data
def model_predict(classifier_name):
    # Load model and its weighs from disk
    loaded_file = open(f"{classifier_name}.yaml", "r")
    loaded_model = loaded_file.read()
    loaded_file.close()
    classifier_model = model_from_yaml(loaded_model)

    # Loading weights 
    load_model(f"{classifier_name}.h5")
    print("Loaded model from disk\n\n\n")

    # Test data path
    img_dir = "dataset_test/" # Test directory path

    # Iterate over each test image
    data_path = os.path.join(img_dir, "*g")
    files = glob.glob(data_path)
    
    # Make a prediction and add to results 
    data = []
    results = []

    for f1 in files:
        img = image.load_img(f1, target_size = (64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        data.append(img)
        result = classifier_model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)

    return(results)


In [168]:
# Creating a function for the accuracy
def accuracy_calc(predictions):
    actual_values = [3, 0, 2, 0, 1, 2, 3, 1]
    accuracy = metrics.accuracy_score(actual_values,predictions)
    return(accuracy)


In [169]:
# Creating a function to run each model from the parameter grid
def model_run(parameters):
    model = parameters["model_name"]
    model_fit_save(parameters)
    res = model_predict(model)
    return(accuracy_calc(res))

In [170]:
import io
from concurrent.futures import ThreadPoolExecutor


with ThreadPoolExecutor(1) as executor:
    executor_results =  executor.map(model_run,parameter_grid)
executor_results = list(executor_results)
executor_results

Fitting the model: {'model_name': 'model_1', 'steps_per_epoch': 1, 'epochs': 1}


Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 0.5319 - accuracy: 0.9375
Saving model to disk as a file

Loaded model from disk



Fitting the model: {'model_name': 'model_2', 'steps_per_epoch': 1, 'epochs': 2}


Found 88 images belonging to 4 classes.
Epoch 1/2
1/1 [==============================] - 0s 1ms/step - loss: 0.4794 - accuracy: 0.9062
Epoch 2/2
1/1 [==============================] - 0s 1ms/step - loss: 0.4075 - accuracy: 0.9062
Saving model to disk as a file

Loaded model from disk



Fitting the model: {'model_name': 'model_3', 'steps_per_epoch': 1, 'epochs': 3}


Found 88 images belonging to 4 classes.
Epoch 1/3
1/1 [==============================] - 0s 4ms/step - loss: 0.3561 - accuracy: 0.8750
Epoch 2/3
1/1 [==============================] - 0s 2ms/step - loss: 0.3272 - accuracy: 0.8750
Epoch 3/3
1/1 [==============================] - 0s 1

[0.125, 0.25, 0.5, 0.25, 0.125, 0.625, 0.125, 0.25, 0.5, 0.25]

In [172]:
executor_results

[0.125, 0.25, 0.5, 0.25, 0.125, 0.625, 0.125, 0.25, 0.5, 0.25]

In [173]:
# Lets get the results 
final_results_df = pd.DataFrame(parameter_grid)
final_results_df.insert(3, "Accuracy", executor_results, True) 
final_results_df

,model_name,steps_per_epoch,epochs,Accuracy
0,model_1,1,1,0.125
1,model_2,1,2,0.250
2,model_3,1,3,0.500
3,model_4,2,4,0.250
4,model_5,2,5,0.125
5,model_6,2,6,0.625
6,model_7,3,7,0.125
7,model_8,3,8,0.250
8,model_9,5,9,0.500
9,model_10,5,10,0.250


The accuracy results of all our 10 different models are extremely poor with just 1 ThreadPool Executor

In [174]:
# Lets lets run our 10 models on 5 ThreadPool Executors
with ThreadPoolExecutor(5) as executor:
    executor_results =  executor.map(model_run,parameter_grid)
executor_results = list(executor_results)
executor_results

Fitting the model: {'model_name': 'model_1', 'steps_per_epoch': 1, 'epochs': 1}


Fitting the model: {'model_name': 'model_2', 'steps_per_epoch': 1, 'epochs': 2}


Fitting the model: {'model_name': 'model_3', 'steps_per_epoch': 1, 'epochs': 3}


Fitting the model: {'model_name': 'model_4', 'steps_per_epoch': 2, 'epochs': 4}


Fitting the model: {'model_name': 'model_5', 'steps_per_epoch': 2, 'epochs': 5}


Found 88 images belonging to 4 classes.
Found 88 images belonging to 4 classes.
Found 88 images belonging to 4 classes.Found 88 images belonging to 4 classes.

Found 88 images belonging to 4 classes.
Epoch 1/5
Epoch 1/3
Epoch 1/4
Epoch 1/2
1/1 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 1ms/step - loss: 0.0053 - accuracy: 1.0000
Saving model to disk as a file

2/2 [==============================] - 1s 699ms/step - loss: 0.0038 - accuracy: 1.0000
Loaded model from disk



Epoch 2/2
Epoch 2/5
1/1 [

[0.25, 0.375, 0.25, 0.25, 0.25, 0.25, 0.375, 0.25, 0.25, 0.25]

In [176]:
# Lets get the results 
final_results_df = pd.DataFrame(parameter_grid)
final_results_df.insert(3, "Accuracy", executor_results, True) 
final_results_df

,model_name,steps_per_epoch,epochs,Accuracy
0,model_1,1,1,0.250
1,model_2,1,2,0.375
2,model_3,1,3,0.250
3,model_4,2,4,0.250
4,model_5,2,5,0.250
5,model_6,2,6,0.250
6,model_7,3,7,0.375
7,model_8,3,8,0.250
8,model_9,5,9,0.250
9,model_10,5,10,0.250


Again our results are poor even with 5 different ThreadPool Executors

Now lets try a different approach instead of running all 10 image classification models in parallel. We will run all 10 different individually and separately.

In [180]:
# Writing a function to fit all 10 different models
def fit_model(mod, steps, epo):
    mod.fit(train_datagen.flow_from_directory('dataset_train', target_size=(64, 64), batch_size = 32,
                    class_mode = 'categorical', seed = 69), steps_per_epoch=steps, epochs=epo)
    return mod

In [181]:
# Creating a function to save the model onto the disk

def model_disk_save(m):
    m_yaml = m.to_yaml()
    m_name = m
    with open(f"{m}.yaml", "w") as f:
        f.write(m_yaml)
        m.save(f"{m}.h5")
        print("Saving model to disk as a file\n")

In [184]:
model_1 = fit_model(classifier, 1, 1)
model_disk_save(model_1)
model_1_pred = model_predict(model_1)
accuracy_model_1 = accuracy_calc(model_1_pred)
accuracy_model_1

Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 2.1173e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.25

In [185]:
model_2 = fit_model(classifier, 1, 2)
model_disk_save(model_2)
model_2_pred = model_predict(model_2)
accuracy_model_2 = accuracy_calc(model_2_pred)
accuracy_model_2

Found 88 images belonging to 4 classes.
Epoch 1/2
1/1 [==============================] - 0s 1ms/step - loss: 2.5453e-04 - accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 813us/step - loss: 1.4549e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.375

In [186]:
model_3 = fit_model(classifier, 1, 3)
model_disk_save(model_3)
model_3_pred = model_predict(model_1)
accuracy_model_3 = accuracy_calc(model_3_pred)
accuracy_model_3

Found 88 images belonging to 4 classes.
Epoch 1/3
1/1 [==============================] - 0s 1ms/step - loss: 1.8435e-04 - accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 1ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 878us/step - loss: 5.3897e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.25

In [187]:
model_4 = fit_model(classifier, 2, 4)
model_disk_save(model_4)
model_4_pred = model_predict(model_4)
accuracy_model_4 = accuracy_calc(model_4_pred)
accuracy_model_4

Found 88 images belonging to 4 classes.
Epoch 1/4
2/2 [==============================] - 0s 102ms/step - loss: 2.1439e-04 - accuracy: 1.0000
Epoch 2/4
2/2 [==============================] - 0s 98ms/step - loss: 1.9956e-04 - accuracy: 1.0000
Epoch 3/4
2/2 [==============================] - 0s 100ms/step - loss: 1.2906e-04 - accuracy: 1.0000
Epoch 4/4
2/2 [==============================] - 0s 99ms/step - loss: 5.0428e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.25

In [188]:
model_5 = fit_model(classifier, 2, 5)
model_disk_save(model_5)
model_5_pred = model_predict(model_5)
accuracy_model_5 = accuracy_calc(model_5_pred)
accuracy_model_5

Found 88 images belonging to 4 classes.
Epoch 1/5
2/2 [==============================] - 0s 105ms/step - loss: 1.6831e-04 - accuracy: 1.0000
Epoch 2/5
2/2 [==============================] - 0s 136ms/step - loss: 3.0468e-04 - accuracy: 1.0000
Epoch 3/5
2/2 [==============================] - 0s 134ms/step - loss: 4.3742e-04 - accuracy: 1.0000
Epoch 4/5
2/2 [==============================] - 0s 96ms/step - loss: 2.1983e-04 - accuracy: 1.0000
Epoch 5/5
2/2 [==============================] - 0s 95ms/step - loss: 1.5012e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.25

In [189]:
model_6 = fit_model(classifier, 2, 6)
model_disk_save(model_6)
model_6_pred = model_predict(model_6)
accuracy_model_6 = accuracy_calc(model_6_pred)
accuracy_model_6

Found 88 images belonging to 4 classes.
Epoch 1/6
2/2 [==============================] - 0s 134ms/step - loss: 9.4847e-04 - accuracy: 1.0000
Epoch 2/6
2/2 [==============================] - 0s 104ms/step - loss: 1.9333e-04 - accuracy: 1.0000
Epoch 3/6
2/2 [==============================] - 0s 101ms/step - loss: 3.0439e-04 - accuracy: 1.0000
Epoch 4/6
2/2 [==============================] - 0s 110ms/step - loss: 6.6808e-04 - accuracy: 1.0000
Epoch 5/6
2/2 [==============================] - 0s 142ms/step - loss: 6.3826e-04 - accuracy: 1.0000
Epoch 6/6
2/2 [==============================] - 0s 139ms/step - loss: 3.1712e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.25

In [191]:
model_7 = fit_model(classifier, 3, 7)
model_disk_save(model_7)
model_7_pred = model_predict(model_7)
accuracy_model_7 = accuracy_calc(model_7_pred)
accuracy_model_7

Found 88 images belonging to 4 classes.
Epoch 1/7
3/3 [==============================] - 1s 185ms/step - loss: 2.0413e-04 - accuracy: 1.0000
Epoch 2/7
3/3 [==============================] - 1s 186ms/step - loss: 3.2823e-04 - accuracy: 1.0000
Epoch 3/7
3/3 [==============================] - 0s 145ms/step - loss: 2.2351e-04 - accuracy: 1.0000
Epoch 4/7
3/3 [==============================] - 1s 175ms/step - loss: 1.5286e-04 - accuracy: 1.0000
Epoch 5/7
3/3 [==============================] - 0s 150ms/step - loss: 2.4591e-04 - accuracy: 1.0000
Epoch 6/7
3/3 [==============================] - 1s 175ms/step - loss: 2.1253e-04 - accuracy: 1.0000
Epoch 7/7
3/3 [==============================] - 0s 151ms/step - loss: 1.2280e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.25

In [192]:
model_8 = fit_model(classifier, 3, 8)
model_disk_save(model_8)
model_8_pred = model_predict(model_8)
accuracy_model_8 = accuracy_calc(model_8_pred)
accuracy_model_8

Found 88 images belonging to 4 classes.
Epoch 1/8
3/3 [==============================] - 0s 155ms/step - loss: 2.3734e-04 - accuracy: 1.0000
Epoch 2/8
3/3 [==============================] - 0s 153ms/step - loss: 1.5050e-04 - accuracy: 1.0000
Epoch 3/8
3/3 [==============================] - 0s 149ms/step - loss: 2.3864e-04 - accuracy: 1.0000
Epoch 4/8
3/3 [==============================] - 0s 151ms/step - loss: 1.7684e-04 - accuracy: 1.0000
Epoch 5/8
3/3 [==============================] - 0s 155ms/step - loss: 6.9642e-05 - accuracy: 1.0000
Epoch 6/8
3/3 [==============================] - 0s 154ms/step - loss: 1.4227e-04 - accuracy: 1.0000
Epoch 7/8
3/3 [==============================] - 1s 177ms/step - loss: 1.0295e-04 - accuracy: 1.0000
Epoch 8/8
3/3 [==============================] - 0s 147ms/step - loss: 2.4272e-04 - accuracy: 1.0000
Saving model to disk as a file

Loaded model from disk





0.375

In [193]:
# model_9 

n_batch = 5

n_epochs=9

# loop through epochs
for e in range(n_epochs):
    print('Epoch', e)
    batches = 0
      # loop through batches
    for x_batch, y_batch in train_datagen.flow_from_directory('dataset_train/', target_size=(64, 64), batch_size = 32, class_mode = 'categorical', seed = 74):
        classifier.fit(x_batch, y_batch)
        batches += 1
        if batches >= n_batch:
        # we need to break the loop by hand because the generator loops indefinitely
            break
            
model_9 = classifier
model_disk_save(model_9)
model_9_pred = model_predict(model_9)
accuracy_model_9 = accuracy_calc(model_9_pred)
accuracy_model_9

Epoch 0
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 8.5361e-05 - accuracy: 1.0000
Epoch 1
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 8.3109e-05 - accuracy: 1.0000
Epoch 2
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 988us/step - loss: 8.0351e-05 - accuracy: 1.0000
Epoch 3
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 7.7537e-05 - accuracy: 1.0000
Epoch 4
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 7.4839e-05 - accuracy: 1.0000
Epoch 5
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 7.2281e-05 - accuracy: 1.0000
Epoch 6
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 971us/step - loss: 6.9895e-05 - accuracy: 1.0000
Epoch 7
Found 88 images belonging to 

0.125

In [194]:
# model_10

n_batch = 5

n_epochs=10

# loop through epochs
for e in range(n_epochs):
    print('Epoch', e)
    batches = 0
      # loop through batches
    for x_batch, y_batch in train_datagen.flow_from_directory('dataset_train/', target_size=(64, 64), batch_size = 32, class_mode = 'categorical', seed = 74):
        classifier.fit(x_batch, y_batch)
        batches += 1
        if batches >= n_batch:
        # we need to break the loop by hand because the generator loops indefinitely
            break
            
model_10 = classifier
model_disk_save(model_10)
model_10_pred = model_predict(model_10)
accuracy_model_10 = accuracy_calc(model_10_pred)
accuracy_model_10

Epoch 0
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 6.3816e-05 - accuracy: 1.0000
Epoch 1
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 6.2099e-05 - accuracy: 1.0000
Epoch 2
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 6.0506e-05 - accuracy: 1.0000
Epoch 3
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 5.8968e-05 - accuracy: 1.0000
Epoch 4
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 5.7524e-05 - accuracy: 1.0000
Epoch 5
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 5.6191e-05 - accuracy: 1.0000
Epoch 6
Found 88 images belonging to 4 classes.
1/1 [==============================] - 0s 1ms/step - loss: 5.4928e-05 - accuracy: 1.0000
Epoch 7
Found 88 images belonging to 4 cl

0.25

In [195]:
# Now lets compare the final results

compare_results_df = pd.DataFrame(parameter_grid)
Accuracy_list = [accuracy_model_1,accuracy_model_2,accuracy_model_3,accuracy_model_4,accuracy_model_5,accuracy_model_6,
    accuracy_model_7,accuracy_model_8,accuracy_model_9,accuracy_model_10]
compare_results_df.insert(3, "Accuracy", Accuracy_list, True) 
compare_results_df

,model_name,steps_per_epoch,epochs,Accuracy
0,model_1,1,1,0.250
1,model_2,1,2,0.375
2,model_3,1,3,0.250
3,model_4,2,4,0.250
4,model_5,2,5,0.250
5,model_6,2,6,0.250
6,model_7,3,7,0.250
7,model_8,3,8,0.375
8,model_9,5,9,0.125
9,model_10,5,10,0.250


So it looks like our results turned out to be same as the ThreadPool executor approach that leveraged parallel processing

# Conceptual Questions: 

4. Discuss the effect of the following on accuracy and loss (train & test): 

    Increasing the steps_per_epoch
    Increasing the number of epochs

Increasing the steps_per_epoch : Steps per epoch refers to the magnitude or the number of iterations of batches before a single epoch of the training process ends or gets completed. steps_per_epoch is all about updating the weights in each iteration through the process of gradient descent. Computation is increased to a great extent wity the increase in the steps_per_epoch parameter value because it involves the update on the weight values and it also leads to the increase in time it takes to complete each epoch . The loss function value is minimized to a great extent as we impose more updates on the weight values involving more number of iterations. The lowering of the loss function ultimately leads to the increase in the accuracy of the predictions which decreases the bias of the model.As we know considering the bias-variance tradeoff that increase in accuracy/decrease in the bias comes with the consequence or expense of increase in the variance of the model and potentially causing the overfitting of the model which then needs to the generalized through dropout mechanisms if we use neural nets.


Increasing the number of epochs :  If we take a particular neural network into consideration, an epoch in such a case is considered to be completed or finished when the entirety of the input training dataset is passed both backward and forward through the entire neural network just once. During such each process, an epoch refers to a hyperparameter that indicates how many times all of the training vectors are used once in order to update the weights of the nodes in the neural network or the internal model parameters. Epoch indicates the number of compelete passes through the entire training dataset. Accuracy of the model predictions increase with the increase in the number of epochs which in turn leads to the decrease in the model errors or the bias. However for the validation set, a lowest point of global minima is reached for the model errors beyond which the error values increases again . This usually happens when the predictive model starts to overfit and incorporates the noise instead of the general patterns within the training data. So this overfit cannot perform accurately when it is exposed to the new unseen data. Early stopping is a very effective strategy or technique to stop this overfitting where the validation set error value reaches the global minimum so that we dont have to over-train our model.

5. Name two uses of zero padding in CNN.

Zero padding is a technique that is widely used in the image classification which involves surrounding a particular matrix of a convolutional neural network with a bunch of zeroes. This technique helps in preserving the image features that exist at the edges of the original matrix which in turn helps in controlling the size of the output feature map. Padding in general works by extending the area that actually processes the image in the receptive field of the convolutional neural network. Some of the advantages of the zero padding are as follows :

Zero padding helps us to preserve the original input size.Zero padding involves adding a border or layer of pixels with zero value around the edges of the input training/test images. Zero padding helps in retaining more important information at the borders more accurately for analysis. In convolutional neural networks in particular , a kernel is a filter within neural networks which slides or moves across the entire image being part of the receptive field, thereby scanning the every pixel of the image and then converts the pixel data into smaller or larger format. The zero padding helps in assisting the kernel in the processing of the image which involves the padding being added to the frame of the image in order to facilitate or allow the kernel to cover the image for processing. It is very likely and easy for the pixels in the corners of the input image to get ignored by the kernel and features or the pixels at the center would dominate the output or those center pixels could repeatedly contribute to the output feature map. The pixels on the border could easily be washed away or ignored very quickly as the volume size reduces after each convolutional layer by a small amount when we dont stack a few layers of zero padding in the convolutional neural network. A much better performance in terms of model accuracy could be obtained by our convolutional neural network by including the technique of zero padding which makes sure that the border pixels of the input image are not lost.

Zero padding helps in building deeper networks that process the input images on a much deeper level.Spatial size of the output feature map could also be dramaticaly controlled using zero padding. The spatial size of the output volume is controlled by or dominated by a few factors which include the size of the size of the receptive field, the amount of zero padding , volume size of the input, the stride ( distance between two consecutive receptive fields) . The size of the spatial output volume is directly proportional to the amount of zero padding layers which helps massively in reducing the output volume size too quickly.


6. What is the use of a 1 x 1 kernel in CNN? 


One of the major use of the 1 x 1 kernel within the realm of the convlutioinal neural networks is the dimensionality reduction or augmentation by reducing the paremeter map. We all know by now that a convolutional layer with a 1 x 1 filter can be helpful in CNN to have control over the number of input feature maps. A simple 1 x 1 filter can be used to summarize the input feature maps. When we stack these 1 x 1 filters and make use of multiple 1 x 1 filters , the tuning of the number of sumaries of the input feature maps can be accomplished which in turn significantly allows the depth of the feature maps to be increased or decreased as needed.

A 1 x 1 kernel helps in mappping the pixels in the input to the output pixel to the next layers with all its channels. These type of simple 1 x 1 kernels are computationally least expensive and are also efficient in their operational functionality.

It also helps in building a deeper network. An effective down-sampling could happen in a 1 x 1 kernel leading to smaller feature vectors  or reduced amount of feature parameters.Identity shortcuts can be directly used as the 1 x 1 kernel restores the original dimensions to match the dimension of the input.


Non-linearity could be added additionally to the neural network using these 1 x 1 kernels.


A 1 x 1 kernel could also increase the accuracy to a great extent by creating a smaller convolutional neural network.In most of the case for the CNNs to have higher accuracy , they are required to have huge number of complex parameters which cause a massive increase in the training time and requires large amount of computational resources.To overcome these kind of drawbacks , its better to have a simple 1 x 1 kernel which include a squeeze layer and an expansion layer to come up with the same level of accuracy as other complex models.

7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?


Since CNN is location invariant , it can really help with the aigility in pattern recognition.The CNN can recognize the pattern of an image or video in any other location once it has recognized the pattern in one location. This is the main dominating property of CNN called the feature extraction. On the other hand , when a DNN recognizes a pattern in one location , it still remains limited in its functiionality and can recognize that kind of a pattern only in that particular location.

Weight sharing is another main advantage of using a CNN which reduces the number of parameters leading to increased efficiency.The same parameters are shared between all the neurans within a CNN thereby leading to the reduction of additional parameters that are needed by the CNN model which makes it a lot more flexible compared to the DNN model.
As a consequence , CNNs also leverage the advantage of using local spatial coherence in the images from the input which allows the to have fewer weights because some of the parameter weights are shared. This is what enables them to make heavy computations at a low cost and they are well suited to extract any information in the form of text , images , voice , video etc

CNNs also have pooling layers which helps a lot in down scaling the image. This happens because the information is retained throughout the network once its extracted and then features are organized spatially identically to the actual image leading to the downscaling of the image by actually reducing the size of the image.

There is no sense of order of the inputs in traditional feedforward neural networks.Shuffling of the images in multiple ways does not impact the performance of the feedforward neural network.Where as CNN taking advantage of local spatial coherence , they can dramatically reduce the computation required in terms of operations needed to process the image by imposing convolutions on the patches of adjacent pixels because adjacent pixels are meaningful.